# Step 4: Baseline Model Design

In this notebook, we define and instantiate the **Baseline Model**. 
This model serves as a benchmark for the project. In Phase 2, we will compare this simple architecture against advanced Transfer Learning models (like ResNet50).

In [1]:
# --- Setup ---
import torch
import sys
import os

# Add src to path
sys.path.append(os.path.abspath('../src'))
from models import SimpleCNN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### 1. Model Architecture Definition
We use a standard Convolutional Neural Network (CNN) with 3 convolutional blocks followed by fully connected layers.

In [2]:
# Initialize Model
# Note: WikiArt typically has 27 styles, but we can detect this dynamically later.
model = SimpleCNN(num_classes=27).to(device)

print(model)

SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=100352, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=27, bias=True)
)


### 2. Scientific Justification (For Report)

**Why this architecture?**
1.  **Simplicity & Speed:** The `SimpleCNN` has significantly fewer parameters than modern deep networks. This allows for rapid iteration and debugging of the training pipeline in Phase 1.
2.  **Benchmarking:** By starting with a shallow network (3 layers), we establish a **lower-bound performance**. This scientifically proves the necessity of deeper networks and Transfer Learning in Phase 2. If this simple model achieves 30% accuracy, and ResNet achieves 80%, we can quantify the value of "Depth" and "Pre-training".
3.  **Resource Constraints:** Training this model from scratch is feasible on mid-range GPUs (like RTX 4060) without requiring massive pre-training data.

### 3. Smoke Test (Forward Pass)
We pass a random tensor through the model to ensure dimensions are calculated correctly and there are no runtime errors.

In [3]:
# Create a dummy input batch: (Batch=4, Channels=3, Height=224, Width=224)
dummy_input = torch.randn(4, 3, 224, 224).to(device)

try:
    output = model(dummy_input)
    print("✅ Forward pass successful.")
    print(f"Input Shape: {dummy_input.shape}")
    print(f"Output Shape: {output.shape} (Should be [4, 27])")
except Exception as e:
    print(f"❌ Error in forward pass: {e}")

✅ Forward pass successful.
Input Shape: torch.Size([4, 3, 224, 224])
Output Shape: torch.Size([4, 27]) (Should be [4, 27])


### 4. Parameter Count
Calculating the complexity of our baseline.

In [4]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Trainable Parameters: {total_params:,}")

Total Trainable Parameters: 51,488,283
